In [2]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import BalancedRandomForestClassifier

from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [5]:
columns = ["obj_ID",'alpha','delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'redshift',
       'plate', 'MJD', 'fiber_ID']
target = ["class"]

In [ ]:
# Create engine
engine = create_engine("sqlite:///

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [8]:
# Load the data
file_path = Path('sample_data.csv', index=False)
df = pd.read_csv(file_path)
# df = df.loc[:, columns].copy()
df

,Unnamed: 0,obj_id,alpha,delta,u,g,r,i,z,run_id,rerun_id,cam_col,field_id,spec_obj_id,class,redshift,plate,mjd,fiber_id
0,0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1.237666e+18,348.335815,25.685581,23.93912,22.92743,21.78178,20.69582,19.83773,4822,301,3,437,8.675069e+18,GALAXY,0.686254,7705,57332,35
96,96,1.237671e+18,42.048515,32.466019,21.80150,18.61152,17.18664,16.54847,16.14959,5934,301,4,155,3.614186e+18,STAR,0.000037,3210,54876,173
97,97,1.237662e+18,169.417236,7.719850,25.08139,22.03635,21.83119,21.71758,21.24758,3841,301,5,31,6.045073e+18,STAR,0.000266,5369,56272,422
98,98,1.237662e+18,243.498908,4.309792,21.23949,20.39086,20.35097,20.25421,19.90627,3910,301,4,222,5.412459e+18,QSO,2.290692,4807,55687,940


In [9]:
# Create our features
X = df.drop('class', axis=1)

# Create our target
y = df['class']

In [10]:
X.describe()

,Unnamed: 0,obj_id,alpha,delta,u,g,r,i,z,run_id,rerun_id,cam_col,field_id,spec_obj_id,redshift,plate,mjd,fiber_id
count,100.000000,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,100.00000
mean,49.500000,1.237666e+18,190.916135,22.497345,22.867176,21.199389,20.084953,19.422210,18.998426,4864.340000,301.0,3.550000,172.130000,6.113754e+18,0.575204,5429.990000,55770.300000,466.61000
std,29.011492,8.299307e+12,104.182062,16.340209,1.873193,1.561941,1.589658,1.500648,1.535673,1932.349474,0.0,1.250252,115.109053,3.229987e+18,0.597542,2868.790948,1716.179763,277.20954
min,0.000000,1.237652e+18,1.494389,-8.043402,18.751240,17.356690,16.497470,15.977110,15.544610,1412.000000,301.0,1.000000,12.000000,4.234019e+17,-0.000608,376.000000,51929.000000,10.00000
25%,24.750000,1.237661e+18,132.899400,7.680188,21.719780,20.371858,19.008765,18.507205,17.992513,3688.500000,301.0,3.000000,91.250000,4.299382e+18,0.151202,3818.500000,55498.750000,222.25000
50%,49.500000,1.237663e+18,180.671973,22.094394,22.781610,21.486810,20.320610,19.489825,18.966610,4059.500000,301.0,4.000000,152.000000,5.652657e+18,0.482437,5020.500000,55893.500000,426.00000
75%,74.250000,1.237674e+18,252.680626,32.785846,24.407297,22.319085,21.105117,20.323055,19.850908,6573.250000,301.0,4.000000,223.500000,8.237289e+18,0.686348,7316.000000,56710.000000,742.00000
max,99.000000,1.237681e+18,357.151519,67.747450,27.097100,24.893950,24.802600,23.178280,22.826470,8157.000000,301.0,6.000000,518.000000,1.278691e+19,2.689002,11357.000000,58522.000000,997.00000


In [12]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'GALAXY': 51, 'STAR': 51, 'QSO': 51})

In [17]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5925925925925926